In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sbn
import cvxopt
from sklearn.model_selection import train_test_split

def polynomial_kernel(x,y,p=3):
  return (1+np.dot(x,y))**p

def gaussian_kernel(x, y, sigma=5.0):
    return np.exp(-np.linalg.norm(x-y)**2 / (2 * (sigma ** 2)))

def generatedataset(size,mean1,mean2,mean3,mean4,cov): 
  np.random.seed(0) 
  n = int(size/4)
  Y1 = np.ones(2*n)*-1.0
  x1 = np.random.multivariate_normal(mean1,cov,size=n)
  x2 = np.random.multivariate_normal(mean3,cov,size=n)
  X1 = np.concatenate((x1,x2),axis=0)
  D1 = np.c_[X1,Y1]
  
  Y2 = np.ones(2*n)
  x1 = np.random.multivariate_normal(mean2,cov,size=n)
  x2 = np.random.multivariate_normal(mean4,cov,size=n)
  X2 = np.concatenate((x1,x2),axis=0)
  D2 = np.c_[X2,Y2]
  dataset = np.concatenate((D1,D2),axis=0)
  np.random.shuffle(dataset)
  return dataset

def HardMkernelSVM(learnset):
  n = len(learnset)
  X = learnset[:,:-1]
  y = learnset[:,-1]
  K = np.zeros((n,n))
  for i in range(n):
    for j in range(n):
      K[i,j] = polynomial_kernel(X[i],X[j])
  P = cvxopt.matrix(np.outer(y,y) * K)
  q = cvxopt.matrix(np.ones(n) * -1)
  A = cvxopt.matrix(y, (1,n))
  b = cvxopt.matrix(0.0)
  G = cvxopt.matrix(np.diag(np.ones(n) * -1))
  h = cvxopt.matrix(np.zeros(n))
  solution = cvxopt.solvers.qp(P, q, G, h, A, b)
  aa = np.ravel(solution['x'])
  sv = aa > 1e-5
  ind = np.arange(len(aa))[sv]
  a = aa[sv]
  SV = X[sv]
  SV_y = y[sv]
  print("%d support vectors out of %d points" % (len(a), n))

  b = 0
  for i in range(len(a)):
    b = b + SV_y[i]
    b = b - np.sum(a*SV_y*K[ind[i],sv])
  b = b/len(a)

  return a,b,SV,SV_y

def predict(a,b,SV,SV_y,testset):
  data = testset[:,:-1]
  lbl = np.zeros(len(data))
  for i in range(len(data)):
    s = 0
    for aa,sv_y,sv in zip(a,SV_y,SV):
      s = s + aa*sv_y*polynomial_kernel(data[i],sv)
    lbl[i] = s
  sbn.scatterplot(data[:,0],data[:,1],hue = np.sign(lbl),palette = ['red','blue']).set_title('Prediction')
  plt.show()
  return np.sign(lbl+b)

def plotmargin(learnset,a,b,SV,SV_y):  
  sbn.scatterplot(learnset[:,0],learnset[:,1],hue = learnset[:,2],palette = ['red','blue'])#.set_title('Prediction')
  sbn.scatterplot(SV[:,0],SV[:,1],hue = SV_y,palette = ['green','brown']).set_title('Learning Dataset with Margin')
  X1, X2 = np.meshgrid(np.linspace(-6,6,50),np.linspace(-6,6,50))
  X = np.array([[x1, x2] for x1,x2 in zip(np.ravel(X1),np.ravel(X2))])
  lb = np.zeros(len(X))
  for i in range(len(X)):
    s = 0
    for aa,sv_y,sv in zip(a,SV_y,SV):
      s = s + aa*sv_y*polynomial_kernel(X[i],sv)
    lb[i] = s
  Z = lb.reshape(X1.shape)
  plt.contour(X1, X2, Z, [0.0], colors = 'k', linewidths = 1, origin = 'lower')
  plt.contour(X1, X2, Z+1, [0.0], colors = 'yellow', linewidths = 1, origin = 'lower')
  plt.contour(X1, X2, Z-1, [0.0], colors = 'yellow', linewidths = 1, origin = 'lower')
  plt.axis("tight")
  plt.show()

def Error(testset,lst):
  c = 0
  n = len(lst)
  for i in range(n):
    if testset[i][2] != lst[i]:
      c = c+1
  return c*100/n;

size = 1000
mean1 = [-1, 2]
mean2 = [1, -1]
mean3 = [4, -4]
mean4 = [-4, 4]
cov = [[1.0,0.8], [0.8, 1.0]]
dataset = generatedataset(size,mean1,mean2,mean3,mean4,cov)
sbn.scatterplot(dataset[:,0],dataset[:,1],hue = dataset[:,2],palette = ['red','blue']).set_title('Generated Dataset')
plt.show()

learnset,testset = train_test_split(dataset,test_size = 0.1, random_state=1)
sbn.scatterplot(learnset[:,0],learnset[:,1],hue = learnset[:,2],palette = ['red','blue']).set_title('Learning Dataset')
plt.show()
sbn.scatterplot(testset[:,0],testset[:,1],hue = testset[:,2],palette = ['red','blue']).set_title('Actual Testing Set')
plt.show()

a,b,sv,sy = HardMkernelSVM(learnset)
plotmargin(learnset,a,b,sv,sy)

lst = predict(a,b,sv,sy,testset)
print("Error in prediction : "+str(Error(testset,lst))+"%")